In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Feb 10 10:30:09 2020

@author: hadi
"""
#lesion dataset ---- 8 classes

import keras
from keras.optimizers import SGD
from keras.layers import GlobalAveragePooling2D, Dropout, Dense
import tensorflow as tf
from keras.models import Model
from keras.models import model_from_json
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras import backend as K
from matplotlib import pyplot as plt
from sklearn.metrics import log_loss
from keras.utils import multi_gpu_model
from keras.utils import np_utils
import numpy as np
import sys
sys.setrecursionlimit(1000)
from keras.preprocessing.image import ImageDataGenerator
from timeit import default_timer as timer    
from keras_preprocessing.image import ImageDataGenerator
from numpy import linspace
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from tensorflow.python.client import device_lib
device_lib.list_local_devices()# Check if we're using a GPU device
from sklearn.metrics import classification_report, confusion_matrix
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
import efficientnet.keras as efn 
import efficientnet.tfkeras
from tensorflow.keras.models import load_model

Using TensorFlow backend.


In [2]:
channel = 3; bs = 16 #batch_size
nb_epoch = 10;
img_rows, img_cols = 224, 224; # Resolution of inputs
dirc = '/home/hadi/python/SparseFool/ordered/CFAR_10/dataset/cifar/raw/';#skin lesion directory




datagen = ImageDataGenerator(rescale=1./255.,rotation_range = 90,horizontal_flip=True,vertical_flip = False)
train_generator = datagen.flow_from_directory(
        directory = dirc + '/train',
        target_size=(img_rows, img_cols), color_mode='rgb',
        classes=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'],
        class_mode='categorical', batch_size=bs, shuffle=True, seed=42, subset=None, interpolation='bicubic' )

valid_generator = datagen.flow_from_directory(
        directory=dirc + '/val',
        target_size=(img_rows, img_cols), color_mode='rgb',
        classes=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'],
        class_mode='categorical',batch_size=bs, shuffle=True, seed=42,subset=None, interpolation='bicubic')

datagenTest = ImageDataGenerator(rescale=1./255.,rotation_range = 0,horizontal_flip=False,vertical_flip = False)


# test_generator = datagenTest.flow_from_directory(
#         directory=dirc + '/test',
#         target_size=(img_rows, img_cols), color_mode='rgb',
#         classes=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'],
#         class_mode='categorical',batch_size=bs, shuffle=True, seed=42,subset=None, interpolation='bicubic')

# test_generator_sparsified_50 = datagenTest.flow_from_directory(
#         directory='/home/hadi/python/SparseFool/ordered/CFAR_10/dataset/cifar/sparsified/50Perc/test',
#         target_size=(img_rows, img_cols), color_mode='rgb',
#         classes=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'],
#         class_mode='categorical',batch_size=bs, shuffle=False, seed=42,subset=None, interpolation='bicubic')


# test_generator_sparsified_60 = datagenTest.flow_from_directory(
#         directory='/home/hadi/python/SparseFool/ordered/CFAR_10/dataset/cifar/sparsified/60Perc/test',
#         target_size=(img_rows, img_cols), color_mode='rgb',
#         classes=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'],
#         class_mode='categorical',batch_size=bs, shuffle=False, seed=42,subset=None, interpolation='bicubic')



# test_generator_sparsified_70 = datagenTest.flow_from_directory(
#         directory='/home/hadi/python/SparseFool/ordered/CFAR_10/dataset/cifar/sparsified/70Perc/test',
#         target_size=(img_rows, img_cols), color_mode='rgb',
#         classes=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'],
#         class_mode='categorical',batch_size=bs, shuffle=False, seed=42,subset=None, interpolation='bicubic')

# test_generator_sparsified_80 = datagenTest.flow_from_directory(
#         directory='/home/hadi/python/SparseFool/ordered/CFAR_10/dataset/cifar/sparsified/80Perc/test',
#         target_size=(img_rows, img_cols), color_mode='rgb',
#         classes=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'],
#         class_mode='categorical',batch_size=bs, shuffle=False, seed=42,subset=None, interpolation='bicubic')

##################################Low Frequency
test_generator_sparsified_LoF_50 = datagenTest.flow_from_directory(
        directory='/home/hadi/python/SparseFool/ordered/CFAR_10/dataset/cifar/sparsified/LoF/50Perc/test',
        target_size=(img_rows, img_cols), color_mode='rgb',
        classes=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'],
        class_mode='categorical',batch_size=bs, shuffle=False, seed=42,subset=None, interpolation='bicubic')

test_generator_sparsified_LoF_60 = datagenTest.flow_from_directory(
        directory='/home/hadi/python/SparseFool/ordered/CFAR_10/dataset/cifar/sparsified/LoF/60Perc/test',
        target_size=(img_rows, img_cols), color_mode='rgb',
        classes=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'],
        class_mode='categorical',batch_size=bs, shuffle=False, seed=42,subset=None, interpolation='bicubic')

test_generator_sparsified_LoF_70 = datagenTest.flow_from_directory(
        directory='/home/hadi/python/SparseFool/ordered/CFAR_10/dataset/cifar/sparsified/LoF/70Perc/test',
        target_size=(img_rows, img_cols), color_mode='rgb',
        classes=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'],
        class_mode='categorical',batch_size=bs, shuffle=False, seed=42,subset=None, interpolation='bicubic')

test_generator_sparsified_LoF_80 = datagenTest.flow_from_directory(
        directory='/home/hadi/python/SparseFool/ordered/CFAR_10/dataset/cifar/sparsified/LoF/80Perc/test',
        target_size=(img_rows, img_cols), color_mode='rgb',
        classes=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'],
        class_mode='categorical',batch_size=bs, shuffle=False, seed=42,subset=None, interpolation='bicubic')
#######################High Frequency

test_generator_sparsified_HiF_50 = datagenTest.flow_from_directory(
        directory='/home/hadi/python/SparseFool/ordered/CFAR_10/dataset/cifar/sparsified/HiF/50Perc/test',
        target_size=(img_rows, img_cols), color_mode='rgb',
        classes=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'],
        class_mode='categorical',batch_size=bs, shuffle=False, seed=42,subset=None, interpolation='bicubic')

test_generator_sparsified_HiF_60 = datagenTest.flow_from_directory(
        directory='/home/hadi/python/SparseFool/ordered/CFAR_10/dataset/cifar/sparsified/HiF/60Perc/test',
        target_size=(img_rows, img_cols), color_mode='rgb',
        classes=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'],
        class_mode='categorical',batch_size=bs, shuffle=False, seed=42,subset=None, interpolation='bicubic')

test_generator_sparsified_HiF_70 = datagenTest.flow_from_directory(
        directory='/home/hadi/python/SparseFool/ordered/CFAR_10/dataset/cifar/sparsified/HiF/70Perc/test',
        target_size=(img_rows, img_cols), color_mode='rgb',
        classes=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'],
        class_mode='categorical',batch_size=bs, shuffle=False, seed=42,subset=None, interpolation='bicubic')

test_generator_sparsified_HiF_80 = datagenTest.flow_from_directory(
        directory='/home/hadi/python/SparseFool/ordered/CFAR_10/dataset/cifar/sparsified/HiF/80Perc/test',
        target_size=(img_rows, img_cols), color_mode='rgb',
        classes=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'],
        class_mode='categorical',batch_size=bs, shuffle=False, seed=42,subset=None, interpolation='bicubic')

Found 50000 images belonging to 10 classes.
Found 0 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.


In [4]:


base_model = keras.applications.inception_v3.InceptionV3(include_top=False, weights="imagenet",input_shape=(img_rows, img_cols, 3));  # or weights='noisy-student'
CLASSES = 10
x = base_model.output
x = GlobalAveragePooling2D(name='avg_pool')(x)
x = Dropout(0.4)(x)
predictions = Dense(CLASSES, activation='softmax')(x)
parallel_model = Model(inputs=base_model.input, outputs=predictions)

sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
parallel_model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])



history2 = parallel_model.fit(train_generator,
                steps_per_epoch=int(1000),verbose=1,epochs=nb_epoch,validation_data=valid_generator)


print(test_generator.class_indices)
score = parallel_model.evaluate(test_generator, verbose=1)
print("%s: %.2f%%" % (parallel_model.metrics_names[1], score[1]*100))

W0228 05:24:45.964602 140171016222464 deprecation_wrapper.py:119] From /home/hadi/anaconda3/envs/venv1/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4070: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0228 05:24:46.291880 140171016222464 deprecation_wrapper.py:119] From /home/hadi/anaconda3/envs/venv1/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4074: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.



87916544/87910968 [==============================] - 1s 0us/step


W0228 05:24:59.857266 140171016222464 deprecation_wrapper.py:119] From /home/hadi/anaconda3/envs/venv1/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Epoch 1/10
1000/1000 [==============================] - 151s 151ms/step - loss: 1.0902 - accuracy: 0.6192
Epoch 2/10
1000/1000 [==============================] - 186s 186ms/step - loss: 0.6774 - accuracy: 0.7693
Epoch 3/10
1000/1000 [==============================] - 186s 186ms/step - loss: 0.5869 - accuracy: 0.7981
Epoch 4/10
1000/1000 [==============================] - 195s 195ms/step - loss: 0.4893 - accuracy: 0.8347
Epoch 5/10
1000/1000 [==============================] - 193s 193ms/step - loss: 0.4399 - accuracy: 0.8502
Epoch 6/10
1000/1000 [==============================] - 177s 177ms/step - loss: 0.4170 - accuracy: 0.8579
Epoch 7/10
1000/1000 [==============================] - 178s 178ms/step - loss: 0.3697 - accuracy: 0.8741
Epoch 8/10
1000/1000 [==============================] - 175s 175ms/step - loss: 0.3476 - accuracy: 0.8805
Epoch 9/10
1000/1000 [==============================] - 176s 176ms/step - loss: 0.3425 - accuracy: 0.8817
Epoch 10/10
1000/1000 [=======================

NameError: name 'test_generator' is not defined

In [3]:
model_path = '/home/hadi/python/SparseFool/ordered/CFAR_10/effectOfSparsity/InceptionV3/saved_model/'
load_mode = True
if load_mode:
    parallel_model = keras.models.load_model(model_path + 'InceptionV3_CIFAR10_raw_10.h5')
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    parallel_model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    print('model is loaded')

W0301 00:34:57.621747 139723024750336 deprecation_wrapper.py:119] From /home/hadi/anaconda3/envs/venv1/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4070: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0301 00:34:57.881206 139723024750336 deprecation_wrapper.py:119] From /home/hadi/anaconda3/envs/venv1/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4074: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.

W0301 00:35:12.142500 139723024750336 deprecation_wrapper.py:119] From /home/hadi/anaconda3/envs/venv1/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



model is loaded


In [6]:
score = parallel_model.evaluate(test_generator, verbose=1)
print('raw: ',"%s: %.2f%%" % (parallel_model.metrics_names[1], score[1]*100))

print('---------------------------------------------------------------------')

score = parallel_model.evaluate(test_generator_sparsified_50, verbose=1)
print('50%: ', "%s: %.2f%%" % (parallel_model.metrics_names[1], score[1]*100))

print('---------------------------------------------------------------------')

score = parallel_model.evaluate(test_generator_sparsified_60, verbose=1)
print('60%: ', "%s: %.2f%%" % (parallel_model.metrics_names[1], score[1]*100))

print('---------------------------------------------------------------------')

score = parallel_model.evaluate(test_generator_sparsified_70, verbose=1)
print('70%: ', "%s: %.2f%%" % (parallel_model.metrics_names[1], score[1]*100))

print('---------------------------------------------------------------------')

score = parallel_model.evaluate(test_generator_sparsified_80, verbose=1)
print('80%: ', "%s: %.2f%%" % (parallel_model.metrics_names[1], score[1]*100))

625/625 [==============================] - 26s 41ms/step
raw:  accuracy: 93.27%
---------------------------------------------------------------------
625/625 [==============================] - 24s 39ms/step
50%:  accuracy: 92.60%
---------------------------------------------------------------------
625/625 [==============================] - 24s 38ms/step
60%:  accuracy: 91.84%
---------------------------------------------------------------------
625/625 [==============================] - 24s 38ms/step
70%:  accuracy: 90.32%
---------------------------------------------------------------------
625/625 [==============================] - 24s 38ms/step
80%:  accuracy: 86.05%


In [7]:
model_path = '/home/hadi/python/SparseFool/ordered/CFAR_10/effectOfSparsity/InceptionV3/saved_model/'
parallel_model.save(model_path + 'InceptionV3_CIFAR10_raw_10.h5')
print('model saved')

model saved


In [4]:
score = parallel_model.evaluate(test_generator_sparsified_LoF_50, verbose=1)
print('50%: ', "%s: %.2f%%" % (parallel_model.metrics_names[1], score[1]*100))
print('---------------------------------------------------------------------')

score = parallel_model.evaluate(test_generator_sparsified_LoF_60, verbose=1)
print('60%: ', "%s: %.2f%%" % (parallel_model.metrics_names[1], score[1]*100))
print('---------------------------------------------------------------------')


score = parallel_model.evaluate(test_generator_sparsified_LoF_70, verbose=1)
print('70%: ', "%s: %.2f%%" % (parallel_model.metrics_names[1], score[1]*100))
print('---------------------------------------------------------------------')

score = parallel_model.evaluate(test_generator_sparsified_LoF_80, verbose=1)
print('80%: ', "%s: %.2f%%" % (parallel_model.metrics_names[1], score[1]*100))
print('---------------------------------------------------------------------')


625/625 [==============================] - 30s 47ms/step
50%:  accuracy: 80.83%
---------------------------------------------------------------------
625/625 [==============================] - 27s 43ms/step
60%:  accuracy: 80.84%
---------------------------------------------------------------------
625/625 [==============================] - 27s 43ms/step
70%:  accuracy: 79.40%
---------------------------------------------------------------------
625/625 [==============================] - 27s 43ms/step
80%:  accuracy: 66.98%
---------------------------------------------------------------------


In [5]:
score = parallel_model.evaluate(test_generator_sparsified_HiF_50, verbose=1)
print('50%: ', "%s: %.2f%%" % (parallel_model.metrics_names[1], score[1]*100))
print('---------------------------------------------------------------------')

score = parallel_model.evaluate(test_generator_sparsified_HiF_60, verbose=1)
print('60%: ', "%s: %.2f%%" % (parallel_model.metrics_names[1], score[1]*100))
print('---------------------------------------------------------------------')


score = parallel_model.evaluate(test_generator_sparsified_HiF_70, verbose=1)
print('70%: ', "%s: %.2f%%" % (parallel_model.metrics_names[1], score[1]*100))
print('---------------------------------------------------------------------')

score = parallel_model.evaluate(test_generator_sparsified_HiF_80, verbose=1)
print('80%: ', "%s: %.2f%%" % (parallel_model.metrics_names[1], score[1]*100))
print('---------------------------------------------------------------------')


625/625 [==============================] - 27s 43ms/step
50%:  accuracy: 31.42%
---------------------------------------------------------------------
625/625 [==============================] - 27s 43ms/step
60%:  accuracy: 30.80%
---------------------------------------------------------------------
625/625 [==============================] - 26s 42ms/step
70%:  accuracy: 25.93%
---------------------------------------------------------------------
625/625 [==============================] - 27s 43ms/step
80%:  accuracy: 15.87%
---------------------------------------------------------------------
